In [ ]:
from utils_new import *

Get and load data from Obenhaus et al. (2022)


In [ ]:
pd.set_option('display.max_rows', 10)

data_dir = '/Users/erihe/OneDrive - NTNU/TDA_review/functional'

AngularRate = pd.read_pickle(data_dir + '/AngularRate')
AngularRate_Stats = pd.read_pickle(data_dir + '/AngularRate_Stats')
BorderScore = pd.read_pickle(data_dir + '/BorderScore')
BVScore = pd.read_pickle(data_dir + '/BVScore')
FilteredCells = pd.read_pickle(data_dir + '/FilteredCells')
FilteredSpikes = pd.read_pickle(data_dir + '/FilteredSpikes')
GridScore = pd.read_pickle(data_dir + '/GridScore')
GridScore_Stats = pd.read_pickle(data_dir + '/GridScore_Stats')
MECcells = pd.read_pickle(data_dir + '/MECcells')
NumberCellTypes_MEC = pd.read_pickle(data_dir + '/NumberCellTypes_MEC')
NumberCellTypes_PAS = pd.read_pickle(data_dir + '/NumberCellTypes_PAS')
OVCScore = pd.read_pickle(data_dir + '/OVCScores')
PAScells = pd.read_pickle(data_dir + '/PAScells')
Session = pd.read_pickle(data_dir + '/Session')
Tracking_Linear = pd.read_pickle(data_dir + '/Tracking_Linear')
Tracking_OpenField = pd.read_pickle(data_dir + '/Tracking_OpenField')
TrackingRaw = pd.read_pickle(data_dir + '/TrackingRaw')


Analyze three sessions

In [ ]:
thr = 0.835
n_points = 1200
k = 1000
maxdim = 1   
metric = 'cosine'
sp = 50
eps = 0.5
dim = 6
sig = 2
lenc = 10
metasessions = ['5e192857bec0c163','05304a7b5f0d845a','9eedcb2fc1c0dc44',]


In [ ]:
for ii, i in enumerate(np.array(metasessions)):
    i = metasessions[ii] 
    msesscurr = i
    print(i, ii)
    sess_nameof = ''
    sess_names = Session[Session['metasession_name']==i]['session_name']
    for sess_name0 in sess_names:
        tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
        if (((tname.lower().find('open')>-1) | (tname.lower().find('opf')>-1)) & 
            ~(tname.lower().find('circular')>-1) & 
           ~(tname.lower().find('obj')>-1)):
            sess_nameof = sess_name0  
    sess_name0 = sess_nameof
    cell_id = np.array(FilteredSpikes[FilteredSpikes['session_name'] == sess_name0]['cell_id'])
    filtered_cellids = np.array(FilteredCells[FilteredCells['session_name']==sess_name0]['cell_id'])
    filtered_bool = np.zeros(len(cell_id), dtype = bool)
    for it, i in enumerate(cell_id):
        filtered_bool[it] = i in filtered_cellids
    print(len(cell_id), sum(filtered_bool))

In [ ]:
for ii, i in enumerate(np.array(metasessions)):
    i = metasessions[ii] 
    msesscurr = i
    print(i, ii)
    sess_nameof = ''
    sess_names = Session[Session['metasession_name']==i]['session_name']
    for sess_name0 in sess_names:
        tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
        if (((tname.lower().find('open')>-1) | (tname.lower().find('opf')>-1)) & 
            ~(tname.lower().find('circular')>-1) & 
           ~(tname.lower().find('obj')>-1)):
            sess_nameof = sess_name0  
    sess_name0 = sess_nameof
    cell_id = np.array(FilteredSpikes[FilteredSpikes['session_name'] == sess_name0]['cell_id'])
    filtered_cellids = np.array(FilteredCells[FilteredCells['session_name']==sess_name0]['cell_id'])
    filtered_bool = np.zeros(len(cell_id), dtype = bool)
    for it, i in enumerate(cell_id):
        filtered_bool[it] = i in filtered_cellids
    indskeep1_bu = np.where(filtered_bool)[0]
    gscore =  np.array(GridScore[(GridScore['session_name'] == sess_name0) & 
                              (GridScore['signal_type'] == 'spikes') & 
                       (GridScore['map_params_id'] == 'A')]["gridscore"])
    bscore  = np.array(BorderScore[((BorderScore['session_name'] == sess_name0) &
                                (BorderScore['signal_type'] == 'spikes') &
                                (BorderScore['map_params_id'] == 'A'))]['borderscore'])
    bvscore =np.array(BVScore[((BVScore['session_name'] == sess_name0) &
                              (BVScore['signal_type'] == 'spikes') & 
                              (BVScore['map_params_id'] == 'A') & 
                          (BVScore['bv_field_dect_method'] == 'bvs'))]['bvs'])
    ovscore = np.array(OVCScore[((OVCScore['base_session'] == sess_name0) &
                              (OVCScore['signal_type'] == 'spikes'))]['ovscore'])
    hdmvl = np.array(AngularRate_Stats[(AngularRate_Stats['session_name'] == sess_name0) & (AngularRate_Stats['signal_type'] == 'spikes')]["mvl"])
    hdmu = np.array(AngularRate_Stats[(AngularRate_Stats['session_name'] == sess_name0) & (AngularRate_Stats['signal_type'] == 'spikes')]["angular_mean"])
    hdstd = np.array(AngularRate_Stats[(AngularRate_Stats['session_name'] == sess_name0) & (AngularRate_Stats['signal_type'] == 'spikes')]["angular_std"])
    acorrs = np.array(GridScore[(GridScore['session_name'] == sess_name0) &
              (GridScore['signal_type'] == 'spikes') & 
              (GridScore['map_params_id'] == 'B')
             ]['acorr'])
    acorrs_all = np.zeros((len(acorrs), len(acorrs[0])**2))
    acorrs1 = np.zeros((len(acorrs), np.shape(acorrs[0])[0], np.shape(acorrs[0])[1]))
    for i in range(len(acorrs)):
        acorrs_all[i,:] = acorrs[i].flatten()
        acorrs1[i,:, :] = acorrs[i]


    Xcorr = {}
    spkall = {}
    xx_all = {}
    yy_all = {}
    tt_all = {}
    hd_all = {}
    for sess_name0 in sess_names:
        tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
        xx0, yy0, speed0, hd0, tt0 = [], [], [], [], []
        if (tname.lower().find('wheel')>-1):# | (tname.lower().find('circular')>-1):
            if len(Tracking_Linear[Tracking_Linear['session_name'] == sess_name0])>0:
                for i in Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['lap_pos']:
                    xx0.extend([i])
                xx0 = np.array(xx0[0])
                yy0 = np.zeros(len(xx0))
                for i in Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['speed']:
                    speed0.extend([i])
                speed0 = np.array(speed0[0])
                hd0 = np.zeros(len(xx0))
                for i in Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['timestamps']:
                    tt0.extend([i])
                tt0 = np.array(tt0[0]) 
        else:
            if len(Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0])>0:
                for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['x_pos']:
                    xx0.extend([i])
                xx0 = np.array(xx0[0])
                for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['y_pos']:
                    yy0.extend([i])
                yy0 = np.array(yy0[0])
                for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['speed']:
                    speed0.extend([i])
                speed0 = np.array(speed0[0])
                for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['head_angle']:
                    hd0.extend([i])
                hd0 = np.array(hd0[0])
                for i in Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['timestamps']:
                    tt0.extend([i])
                tt0 = np.array(tt0[0])
        xx_all[sess_name0] = xx0.copy()
        yy_all[sess_name0] = yy0.copy()
        tt_all[sess_name0] = tt0.copy()
        hd_all[sess_name0] = hd0.copy()

    speed_all = {}
    for sess_name0 in sess_names:
        xspline = CubicSpline(tt_all[sess_name0], gaussian_filter1d(xx_all[sess_name0], 10))
        xx = xspline(tt_all[sess_name0])
        yspline = CubicSpline(tt_all[sess_name0], gaussian_filter1d(yy_all[sess_name0], 10))
        yy = yspline(tt_all[sess_name0])
        speed = np.sqrt(np.square(np.diff(xx)) + np.square(np.diff(yy)))
        speed = np.concatenate(([speed[0]],speed)) # m/10ms
        speed_all[sess_name0] = speed*100

    spk0_all = {}
    for sess_name0 in sess_names:
        spikes0 = []
        for it, i in enumerate(FilteredSpikes[FilteredSpikes['session_name'] == sess_name0]['filtered_spikes']):
            spikes0.append(i)
            it += 1
        spk0_all[sess_name0] = np.array(spikes0).T       

    for sess_name0 in xx_all:
        if len(xx_all[sess_name0]) == 0:
            xx_all[sess_name0] = np.zeros(len(spk0_all[sess_name0].copy()))
            yy_all[sess_name0] = np.zeros(len(spk0_all[sess_name0].copy()))
            speed_all[sess_name0] = np.ones(len(spk0_all[sess_name0].copy()))
            tt_all[sess_name0] = np.arange(len(spk0_all[sess_name0].copy()))
            hd_all[sess_name0] = np.zeros(len(spk0_all[sess_name0].copy()))


    for sess_name0 in sess_names:
        lenspk, num_neurons = np.shape(spk0_all[sess_name0])           
        spikes1 = preprocessing.minmax_scale(spk0_all[sess_name0], axis = 0)
        spikes1 = np.sqrt(gaussian_filter1d(spikes1, axis = 0, sigma = sig))
        Xcorr[sess_name0] = cross_corr_dist(spikes1, lencorr = lenc)
        print('sig, lenc', sig, lenc)
        Xcorr[sess_name0][np.isnan(Xcorr[sess_name0])] = 0    

    dd1 =np.zeros((len(indskeep1_bu), len(indskeep1_bu)))
    for i, sess in enumerate(Xcorr):
        xx1 = Xcorr[sess].copy()
        xx1 = xx1[indskeep1_bu,:]
        xx1 = xx1[:,indskeep1_bu]
        xx1 = np.square(xx1)
        np.fill_diagonal(xx1,0)
        xx1 = squareform(pdist(xx1, 'correlation'))
        dd1 += xx1
    dd1 /= len(Xcorr)
    ind1 = get_ind(dd1,thr, linkage = 'average', bPlot = False)
    
    
    fig = plt.figure(dpi = 300)
    d2 = dd1.copy()
    d2 = d2[np.argsort(ind1), :]
    d2 = d2[:,np.argsort(ind1)]    
    plt.imshow(d2, 
               vmin = np.percentile(d2.flatten(),5),
               vmax = np.percentile(d2.flatten(),95),
              )
    plt.axis('off')

    plt.savefig('Figures/obenhaus_corrmat' + str(ii) + '.png', transparent = True,  bbox_inches='tight', pad_inches=0.2)
    plt.savefig('Figures/obenhaus_corrmat' + str(ii) + '.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)

    data = []
    data_names = []
    for i in range(len(d2)):
        data.append(pd.Series(d2[:,i]))
        data_names.extend(['col_' + str(i)])
    df = pd.concat(data, ignore_index=True, axis=1)            
    df.columns = data_names
    df.to_excel('Source_data/Fig2b_crossmat'+ str(ii) + '.xlsx', sheet_name='crossmat')  

    
    bin_ind = np.bincount(ind1)
    numneuronsind = np.flip(np.argsort(bin_ind))
    print('num: ', bin_ind[numneuronsind[:10]])
    print('ind: ', numneuronsind[:10])
    print('')

    numinds = np.bincount(ind1)
    rel_inds = np.where((numinds>=100))[0]                             
    np.savez('Results/' + tname + '_'+ sess_name0 + '_inds', 
             ind1 = ind1, 
             indskeep1_bu = indskeep1_bu, 
             rel_inds = rel_inds)

    scores = (('acorr2d', acorrs1),
         ('hdmvl', hdmvl),
           ('bscore', bscore),
           ('bvscore', bvscore),
         )

    for i in rel_inds:
        mod_ind1s = indskeep1_bu[(ind1==i) ]
        print('Mod ', i)
        print('num_neurons ', len(mod_ind1s))
        print(mod_ind1s)
        
        sspk2 = spk0_all[sess_nameof][:,mod_ind1s]
        sspk2 = preprocessing.minmax_scale(sspk2)
        sspk2 = preprocessing.scale(np.sqrt(gaussian_filter1d(sspk2,sigma = sig, axis = 0)))
        movetimes00 = np.arange(0,len(sspk2),1)
        xx1 = xx_all[sess_nameof]
        yy1 = yy_all[sess_nameof]
        times1 = np.round(np.linspace(0, len(xx1)-1, len(sspk2))).astype(int)
        scores_cluster(sspk2,scores,  mod_ind1s,xx1[times1],yy1[times1], spk2 = [], 
                       num_example = 6, dim = min(10, len(mod_ind1s)), bUMAP = False)
        plt.show()
        
        for sess_name0 in sess_names[:]:
            print(sess_name0)
            tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]
            print(tname)
            speed0 = speed_all[sess_name0]
            hd0 = hd_all[sess_name0]
            tt0 = tt_all[sess_name0]
            xx0 = xx_all[sess_name0]
            yy0 = yy_all[sess_name0]
            sspk1 = spk0_all[sess_name0][:,mod_ind1s]
            sspk1 = preprocessing.minmax_scale(sspk1, axis = 0)
            sspk1 = gaussian_filter1d(sspk1,sigma = sig, axis = 0)
            sspk1 = np.sqrt(sspk1)                                
            
            times0 = np.round(np.linspace(0, len(xx0)-1, len(sspk1))).astype(int)
            
            movetimes0 = np.where(speed0[times0] >=sp)[0]
            print('movetimes0', len(movetimes0))
            movetimes0 = movetimes0[np.where(np.sum(sspk1[movetimes0,:],1)>=0)[0]]
            print('movetimes0', len(movetimes0))            

            times = times0[movetimes0]
            
            spk1 = preprocessing.scale(sspk1[movetimes0],axis = 0)
            dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
            dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        
            startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
            movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, 
                                             epsilon = eps, startindex = startindex)
            print('movetimes1', len(movetimes1))
            
            indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                               n_points, 1, metric = 'cosine')[0]
            indstemp = movetimes1[indstemp]
            dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]

            d = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
            thresh = np.max(d[~np.isinf(d)])    
            persistence = ripser(d, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
            n_points = len(indstemp)

            coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, 
                                                                    ph_classes = [0,1], bConsistent = True)
            times  = movetimes0[indstemp].copy()
            dgms = persistence['dgms']


            coords_mod2 = coords_ds_consistent.T*2*np.pi
            inds, inds_label =  get_coord_distribution(coords_mod2, numbins = 50,epsilon = 0.1, metric = 'euclidean', startindex = -1)
            phases_1 = get_phases(sspk1[movetimes0[indstemp],:], coords_mod2, inds, inds_label)  
            pcorr1 = match_phases(coords_mod2, sspk1[movetimes0[indstemp],:], phases_1)

            # -ab
            coords_mod1 = coords_mod2.copy()
            coords_mod1[:,0] = 2*np.pi - coords_mod1[:,0]
            phases_2 = get_phases(sspk1[movetimes0[indstemp],:], coords_mod1, inds, inds_label)  
            pcorr2 = match_phases(coords_mod1, sspk1[movetimes0[indstemp],:], phases_2)
            if np.median(pcorr2)> np.median(pcorr1):
                coords_ds_consistent[:,0] = 1 - coords_ds_consistent[:,0]
                pcorr1 = pcorr2
                phases_1 = phases_2
            print(np.median(pcorr1))

            pcorrsqr = match_phases(coords_mod2, sspk1[movetimes0[indstemp],:], phases_1, bSqr = True)


            fig = plot_barcode(dgms)
            fig,ax = plt.subplots(1,1)
            ax.plot(var_exp[:15], lw = 2.5)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)


            coords_mod1 = get_coords_all(sspk1, 
                                         coords_ds_consistent,
                                         movetimes0,                             
                                         indstemp,
                                         dim = dim, 
                                         spk2 = sspk2,
                                         bPCA = True,
                                         bPred = False)

            coords_mod1 = coords_mod1%(2*np.pi)
            fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
            for c in [0,1,]:
                mtot, __, __, circ  = binned_statistic_2d(xx1[times1][movetimes00],
                                                          yy1[times1][movetimes00],
                                                          coords_mod1[movetimes00,c], 
                                                          statistic = circmean, 
                                                          bins = 50,
                                                          expand_binnumbers = True)

                nans = np.isnan(mtot)
                sintot = np.sin(mtot)
                costot = np.cos(mtot)
                sintot[nans] = np.mean(sintot[~nans])
                costot[nans] = np.mean(costot[~nans])
                sintot = gaussian_filter(sintot,sig)
                costot = gaussian_filter(costot,sig)
                mtot = np.cos(np.arctan2(sintot, costot))
                plt.viridis()
                mtot[nans] = np.nan
                axs1[c].imshow(mtot)
                axs1[c].axis('off')
                axs1[c].set_aspect(1/axs1[c].get_data_ratio())
            plt.show()
            dgms1 = {}
            dgms1[0] = dgms.copy()
            np.savez('Results/' + tname + '_' + sess_name0, 
                ssp1 = sspk1, 
                movetimes0 = movetimes0,
                movetimes1 = movetimes1,
                indstemp = indstemp,
                coords_ds = coords_ds_consistent,
                dgms = dgms1,
                pcorr1 = pcorr1,
                pcorr2 = pcorr2,
                pcorrsqr = pcorrsqr,
                xx0 = xx0,
                yy0 = yy0
               )


Run shuffles on cluster

In [ ]:
for ii, msesscurr in enumerate(metasessions):
    sess_names = Session[Session['metasession_name']==msesscurr]['session_name']
    for ss, sess_name0 in enumerate(sess_names[:]):
        diagrams_roll = {}
        for i in range(1,101):
            f = np.load('Horst_Results/' + sess_name0 + '_dgms_rand' + str(i) + '.npz', allow_pickle = True)
            dgmstmp = f['dgms'][()]
            f.close()
            diagrams_roll[i] = dgmstmp[0]
        f = np.load('Horst_Results/' + sess_name0 + '_dgms.npz', allow_pickle = True)
        dgms = f['dgms'][()][0]
        f.close()
        
    
        plot_barcode(dgms, diagrams_roll = diagrams_roll, percshuf = 99, dpi = 300, SaveSourceDataName = 'obenhaus_barcode'+ str(ii))
        plt.savefig('Figures/obenhaus_barcode'+ str(ii) + '_' + str(ss) + '.png', transparent = True,  bbox_inches='tight', pad_inches=0.2)
        plt.savefig('Figures/obenhaus_barcode'+ str(ii) + '_' + str(ss) + '.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)
        plt.show()

In [ ]:
for ii in range(3):
    icount = ii
    i = metasessions[ii] 
    msesscurr = i
    print(i, ii)
    sess_nameof = ''
    sess_names = Session[Session['metasession_name']==i]['session_name']
    for i in range(3):
        coords_mod1 += 1*i
        coords_mod1 = coords_mod1%(2*np.pi)
    dgms1 = {}
    dgms1[0] = dgms.copy()
    sess_name0 = list(sess_names)[0]

    ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
    f = np.load(ff, allow_pickle = True) 
    sspk1 = f['ssp1']
    xx1 = f['xx0']
    yy1 = f['yy0']
    movetimes0 = f['movetimes0']
    movetimes1 = f['movetimes1']
    indstemp = f['indstemp']
    coords_ds_consistent = f['coords_ds']
    f.close()

    coords_mod1 = get_coords_all(sspk1, 
                                 coords_ds_consistent,
                                 movetimes0,                             
                                 indstemp,
                                 dim = dim, 
                                 bPCA = True,
                                 bPred = False)
    if icount == 0:
        coords_mod1 = np.flip(coords_mod1,1)
    elif icount == 1:
        coords_mod1[:,0] -= np.pi/3*coords_mod1[:,1]
        coords_mod1[:,0] = (coords_mod1[:,0]%(2*np.pi))


    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
    times1 = np.round(np.linspace(0, len(xx1)-1, len(coords_mod1))).astype(int)
    for c in [0,1,]:
        mtot, __, __, circ  = binned_statistic_2d(xx1[times1],
                                                  yy1[times1],
                                                  coords_mod1[:,c], 
                                                  statistic = circmean, 
                                                  bins = 50,
                                                  expand_binnumbers = True)
        nans = np.isnan(mtot)
        sintot = np.sin(mtot)
        costot = np.cos(mtot)
        sintot[nans] = np.mean(sintot[~nans])
        costot[nans] = np.mean(costot[~nans])
        sintot = gaussian_filter(sintot,sig)
        costot = gaussian_filter(costot,sig)
        mtot = np.cos(np.arctan2(sintot, costot))
        plt.viridis()
        mtot[nans] = np.nan
        axs1[c].imshow(mtot)
        axs1[c].axis('off')
        axs1[c].set_aspect(1/axs1[c].get_data_ratio())
        data = []
        data_names = []
        for i in range(len(mtot)):
            data.append(pd.Series(mtot[:,i]))
            data_names.extend(['col_' +str(i)])
        df = pd.concat(data, ignore_index=True, axis=1)            
        df.columns = data_names
        df.to_excel('Source_data/Fig2d_obenhaus_stripes_OF'+ str(icount) +'_c' + str(c) +'.xlsx', sheet_name='obenhaus_stripes_OF' +str(icount))  

    plt.savefig('Figures/obenhaus_stripes_OF'+ str(icount) + '.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
    plt.savefig('Figures/obenhaus_stripes_OF'+ str(icount) + '.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)
    plt.show()
    
    
    
    
    sess_name0 = list(sess_names)[1]

    ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
    f = np.load(ff, allow_pickle = True) 
    sspk2 = f['ssp1']
    movetimes0 = f['movetimes0']
    movetimes1 = f['movetimes1']
    indstemp = f['indstemp']
    coords_ds_consistent = f['coords_ds']
    f.close()
    coords_mod1 = get_coords_all(sspk2, 
                                 coords_ds_consistent,
                                 movetimes0,                             
                                 indstemp,
                                 dim = dim, 
                                 spk2 = sspk1,
                                 bPCA = True,
                                 bPred = False)

    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
    times1 = np.round(np.linspace(0, len(xx1)-1, len(coords_mod1))).astype(int)
    if icount == 0:
        coords_mod1[:,1] += np.pi/2*3*1.
        coords_mod1[:,1] = (coords_mod1[:,1]%(2*np.pi))
        coords_mod1[:,1] += np.pi/3*coords_mod1[:,0]
        coords_mod1[:,1] = (coords_mod1[:,1]%(2*np.pi))
        coords_mod1 = np.flip(coords_mod1,1)
    elif icount == 1:
        coords_mod1[:,0] += 3*np.pi/2
        coords_mod1[:,0] = (coords_mod1[:,0]%(2*np.pi))
    elif icount == 2:
        coords_mod1[:,1] += np.pi/3*coords_mod1[:,0]
        coords_mod1[:,1] += np.pi
        coords_mod1[:,1] = (coords_mod1[:,1]%(2*np.pi))
        coords_mod1 = np.flip(coords_mod1,1)


    for c in [0,1,]:
        mtot, __, __, circ  = binned_statistic_2d(xx1[times1],
                                                  yy1[times1],
                                                  coords_mod1[:,c], 
                                                  statistic = circmean, 
                                                  bins = 50,
                                                  expand_binnumbers = True)
        nans = np.isnan(mtot)
        sintot = np.sin(mtot)
        costot = np.cos(mtot)
        sintot[nans] = np.mean(sintot[~nans])
        costot[nans] = np.mean(costot[~nans])
        sintot = gaussian_filter(sintot,sig)
        costot = gaussian_filter(costot,sig)
        mtot = np.cos(np.arctan2(sintot, costot))
        plt.viridis()
        mtot[nans] = np.nan
        axs1[c].imshow(mtot)
        axs1[c].axis('off')
        axs1[c].set_aspect(1/axs1[c].get_data_ratio())
        data = []
        data_names = []
        for i in range(len(mtot)):
            data.append(pd.Series(mtot[:,i]))
            data_names.extend(['col_' +str(i)])
        df = pd.concat(data, ignore_index=True, axis=1)            
        df.columns = data_names
        df.to_excel('Source_data/Fig2d_obenhaus_stripes_W'+ str(icount) + '_c' + str(c) + '.xlsx', sheet_name='obenhaus_stripes_W' +str(icount))  

    plt.savefig('Figures/obenhaus_stripes_W'+ str(icount) + '.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
    plt.savefig('Figures/obenhaus_stripes_W'+ str(icount) + '.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)

    plt.show()
        

In [ ]:
pshifts = np.zeros((3,2))
dim = 6
sig = 1
for icount in range(3):
    msesscurr = metasessions[icount] 
    sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
    sess_nameof = ''
    sess_name0 = list(sess_names)[0]
    ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
    f = np.load(ff, allow_pickle = True) 
    sspk1 = f['ssp1']
    xx1 = f['xx0']
    yy1 = f['yy0']    
    movetimes0 = f['movetimes0']
    movetimes1 = f['movetimes1']
    indstemp = f['indstemp']
    coords_ds_consistent = f['coords_ds']
    f.close()

    coords_mod1 = get_coords_all(sspk1, 
                                 coords_ds_consistent,
                                 movetimes0,                             
                                 indstemp,
                                 dim = dim, 
                                 bPCA = True,
                                 bPred = False)
    
    if icount == 0:
        coords_mod1 = np.flip(coords_mod1,1)
    elif icount == 1:
        coords_mod1[:,0] -= np.pi/3*coords_mod1[:,1]
        coords_mod1[:,0] = (coords_mod1[:,0]%(2*np.pi))
    coords_mod0 = coords_mod1.copy()
    
    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
    times1 = np.round(np.linspace(0, len(xx1)-1, len(coords_mod1))).astype(int)
    for c in [0,1,]:
        mtot, __, __, circ  = binned_statistic_2d(xx1[times1],
                                                  yy1[times1],
                                                  coords_mod1[:,c], 
                                                  statistic = circmean, 
                                                  bins = 50,
                                                  expand_binnumbers = True)
        nans = np.isnan(mtot)
        sintot = np.sin(mtot)
        costot = np.cos(mtot)
        sintot[nans] = np.mean(sintot[~nans])
        costot[nans] = np.mean(costot[~nans])
        sintot = gaussian_filter(sintot,sig)
        costot = gaussian_filter(costot,sig)
        mtot = np.arctan2(sintot, costot)
#        mtot = np.cos(np.arctan2(sintot, costot))
        plt.viridis()
        mtot[nans] = np.nan
        axs1[c].imshow(mtot)
        axs1[c].axis('off')
        axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    plt.show()
    
    
    
    sess_name0 = list(sess_names)[1]

    ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
    f = np.load(ff, allow_pickle = True) 
    sspk2 = f['ssp1']
    movetimes0 = f['movetimes0']
    movetimes1 = f['movetimes1']
    indstemp = f['indstemp']
    coords_ds_consistent = f['coords_ds']
    f.close()
    coords_mod1 = get_coords_all(sspk2, 
                                 coords_ds_consistent,
                                 movetimes0,                             
                                 indstemp,
                                 dim = dim, 
                                 spk2 = sspk1,
                                 bPCA = True,
                                 bPred = False)

    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
    times1 = np.round(np.linspace(0, len(xx1)-1, len(coords_mod1))).astype(int)
    if icount == 0:
        coords_mod1[:,1] += np.pi/2*3*1.
        coords_mod1[:,1] = (coords_mod1[:,1]%(2*np.pi))
        coords_mod1[:,1] += np.pi/3*coords_mod1[:,0]
        coords_mod1[:,1] = (coords_mod1[:,1]%(2*np.pi))
        coords_mod1 = np.flip(coords_mod1,1)
        coords_mod1 = 2*np.pi-coords_mod1
    elif icount == 1:
        coords_mod1[:,0] += 3*np.pi/2
        coords_mod1[:,0] = (coords_mod1[:,0]%(2*np.pi))
        coords_mod1 = 2*np.pi-coords_mod1
    elif icount == 2:
        coords_mod1[:,1] += np.pi/3*coords_mod1[:,0]
        coords_mod1[:,1] += np.pi
        coords_mod1[:,1] = (coords_mod1[:,1]%(2*np.pi))
        coords_mod1 = np.flip(coords_mod1,1)
        coords_mod1[:,1] = 2*np.pi-coords_mod1[:,1]


    pshift = np.arctan2(np.mean(np.sin(coords_mod1 - coords_mod0 ),0), 
                np.mean(np.cos(coords_mod1 - coords_mod0 ),0))
    coords_mod1  = (coords_mod1  - pshift)%(2*np.pi)   

    res = np.mean(np.abs(np.arctan2(np.sin(coords_mod1 - coords_mod0), 
                                    np.cos(coords_mod1 - coords_mod0))))

    print(res)

        
    for c in [0,1,]:
        mtot, __, __, circ  = binned_statistic_2d(xx1[times1],
                                                  yy1[times1],
                                                  coords_mod1[:,c], 
                                                  statistic = circmean, 
                                                  bins = 50,
                                                  expand_binnumbers = True)
        nans = np.isnan(mtot)
        sintot = np.sin(mtot)
        costot = np.cos(mtot)
        sintot[nans] = np.mean(sintot[~nans])
        costot[nans] = np.mean(costot[~nans])
        sintot = gaussian_filter(sintot,sig)
        costot = gaussian_filter(costot,sig)
        mtot = np.arctan2(sintot, costot)
#        mtot = np.cos(np.arctan2(sintot, costot))
        plt.viridis()
        mtot[nans] = np.nan
        axs1[c].imshow(mtot)
        axs1[c].axis('off')
        axs1[c].set_aspect(1/axs1[c].get_data_ratio())
    plt.show()

    
    
    coords_aligned = {}
    sspikes_of1_fis = {}
    movetimes0_fis = {}
    movetimes1_fis = {}
    indstemp_fis = {}
    coords_ds_fis = {}
    dgms_fis = {}
    theta_fis = {}
    
    for sess_name0 in sess_names[:]:
        tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]        

        ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
        f = np.load(ff, allow_pickle = True) 
        sspikes_of1_fis[sess_name0] = f['ssp1']
        movetimes0_fis[sess_name0] = f['movetimes0']
        movetimes1_fis[sess_name0] = f['movetimes1']
        indstemp_fis[sess_name0] = f['indstemp']
        coords_ds_fis[sess_name0] = f['coords_ds']
        pcorr1 = f['pcorr1']
        pcorr2 = f['pcorr2']
        f.close()
        
        


        coords_aligned[sess_name0] = get_coords_all(sspikes_of1_fis[sess_name0],
                                     coords_ds_fis[sess_name0],
                                     movetimes0_fis[sess_name0],                             
                                     indstemp_fis[sess_name0],
                                     dim = 7, 
                                     #spk2 = sspk1,
                                     bPCA = True,
                                     bPred = False)
    sess_name0 = sess_names[0]    
    if icount == 0:
        coords_aligned[sess_name0] = np.flip(coords_aligned[sess_name0],1)
    elif icount == 1:
        coords_aligned[sess_name0][:,0] -= np.pi/3*coords_aligned[sess_name0][:,1]
        coords_aligned[sess_name0][:,0] = (coords_aligned[sess_name0][:,0]%(2*np.pi))
    coords_aligned[sess_name0] = coords_aligned[sess_name0].copy()    
    
    sess_name0 = sess_names[1]
    if icount == 0:
        coords_aligned[sess_name0][:,1] += np.pi/2*3*1.
        coords_aligned[sess_name0][:,1] = (coords_aligned[sess_name0][:,1]%(2*np.pi))
        coords_aligned[sess_name0][:,1] += np.pi/3*coords_aligned[sess_name0][:,0]
        coords_aligned[sess_name0][:,1] = (coords_aligned[sess_name0][:,1]%(2*np.pi))
        coords_aligned[sess_name0] = np.flip(coords_aligned[sess_name0],1)
        coords_aligned[sess_name0] = 2*np.pi-coords_aligned[sess_name0]
    elif icount == 1:
        coords_aligned[sess_name0][:,0] += 3*np.pi/2
        coords_aligned[sess_name0][:,0] = (coords_aligned[sess_name0][:,0]%(2*np.pi))
        coords_aligned[sess_name0] = 2*np.pi-coords_aligned[sess_name0]
    elif icount == 2:
        coords_aligned[sess_name0][:,1] += np.pi/3*coords_aligned[sess_name0][:,0]
        coords_aligned[sess_name0][:,1] += np.pi
        coords_aligned[sess_name0][:,1] = (coords_aligned[sess_name0][:,1]%(2*np.pi))
        coords_aligned[sess_name0] = np.flip(coords_aligned[sess_name0],1)
        coords_aligned[sess_name0][:,1] = 2*np.pi-coords_aligned[sess_name0][:,1]          
    
    coords_aligned[sess_name0]  = (coords_aligned[sess_name0]  - pshift)%(2*np.pi) 
    print(icount)
    np.savez('Results/' + msesscurr + '_coords_aligned' , 
             coords_aligned = coords_aligned, pshift = pshift)            

In [ ]:
for ii in range(3):
    msesscurr = metasessions[ii] 
    sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
    phases_1 = {}
    for sess_name0 in sess_names[:]:
        f = np.load('Results/' + msesscurr + '_coords_aligned.npz' , allow_pickle = True) 
        coords_aligned = f['coords_aligned'][()]
        f.close()
        tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]        

        ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
        f = np.load(ff, allow_pickle = True) 
        sspk1 = f['ssp1']        
        f.close()
        
        coords_bin, inds_label =  get_coord_distribution_binned(coords_aligned[sess_name0], 
                                                                numbins = 50, overlap = 0.)        
        phases_1[sess_name0] = get_phases_binned(sspk1, coords_bin, inds_label)  
        plot_phase_distribution(phases_1[sess_name0], phases_1[sess_name0])
                
    np.savez('Results/' + msesscurr + '_phases',
             phases_1 = phases_1
            )
    plot_phase_distribution(phases_1[sess_names[0]], phases_1[sess_names[1]], SourceName = 'Fig_2f_phasedistr_sess' + str(ii))
    phases_dist_all = {}
    
    num_rand = 1000
    num_neurons = len(phases_1[sess_names[0]]) 
    dist_rand = np.zeros((num_rand, num_neurons))
    dist =  np.sqrt(np.sum(np.square(np.arctan2(np.sin(phases_1[sess_names[0]] - phases_1[sess_names[1]]),
                                  np.cos(phases_1[sess_names[0]] - phases_1[sess_names[1]]))),1))
    for i in range(num_rand):
        np.random.seed(i)
        inds = np.arange(num_neurons)
        np.random.shuffle(inds)
        dist_rand[i,:] =  np.sqrt(np.sum(np.square(np.arctan2(np.sin(phases_1[sess_names[0]] - phases_1[sess_names[1]][inds,:]),
                np.cos(phases_1[sess_names[0]] - phases_1[sess_names[1]][inds,:]))),1))
    plot_cumulative_stat(dist, dist_rand, [0,np.sqrt(2)*np.pi],1, [0,np.sqrt(2)*np.pi], 
                         [0,0.5, 1], [0,np.sqrt(2)*np.pi], [0,1.1], SourceName = '')
#    plot_cumulative_stat(dist, dist_rand, [0,np.sqrt(2)*np.pi],1, [0,np.sqrt(2)*np.pi], 
#                         [0,0.5, 1], [0,np.sqrt(2)*np.pi], [0,1.1])
    plt.show()

In [ ]:
dist = {}
dist_rand = {}

for icount, ii in enumerate([0,1,2]):
    msesscurr = metasessions[ii] 
    sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
    f = np.load('Results/' + msesscurr + '_phases.npz',allow_pickle = True)
    phases_1 = f['phases_1'][()]
    f.close()
    phases_dist_all = {}
    num_rand = 1000
    num_neurons = len(phases_1[sess_names[0]]) 
    dist_rand[icount] = np.zeros((num_rand, num_neurons))
    dist[icount] =  np.sqrt(np.sum(np.square(np.arctan2(np.sin(phases_1[sess_names[0]] - phases_1[sess_names[1]]),
                                  np.cos(phases_1[sess_names[0]] - phases_1[sess_names[1]]))),1))
    for i in range(num_rand):
        np.random.seed(i)
        inds = np.arange(num_neurons)
        np.random.shuffle(inds)
        dist_rand[icount][i,:] =  np.sqrt(np.sum(np.square(np.arctan2(np.sin(phases_1[sess_names[0]] - phases_1[sess_names[1]][inds,:]),
                np.cos(phases_1[sess_names[0]] - phases_1[sess_names[1]][inds,:]))),1))
fig, ax = plt.subplots(1,1, dpi = 120)
for icount, ii in enumerate([4,5,9]):
    plot_cumulative_stat(dist[icount], dist_rand[icount], 
                         [0,np.sqrt(2)*np.pi], 1, 
                         [0,np.sqrt(2)*np.pi], [0,0.5, 1], 
                         [0,np.sqrt(2)*np.pi], [0,1.1],
                        cs = [0,0.2*(icount+1), 0.2*(icount+1)],
                         ax = ax,
                         SourceName='Fig_2g_cumulative_phasedistr' + str(ii)
                        )


In [ ]:
from matplotlib.collections import PathCollection
import cv2 as cv
numbins1 = 30
sig1 = 2
roll = 3
plt.viridis()
icount, ii = 2,2    
msesscurr = metasessions[ii] 
sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
pos_all = {}
spk_all = {}
f = np.load('Results/' + msesscurr + '_coords_aligned.npz' , allow_pickle = True) 
coords_aligned = f['coords_aligned'][()]
f.close()



for sess_name0 in sess_names[:]:
    tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]        
    ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
    f = np.load(ff, allow_pickle = True) 
    spk_all[sess_name0] = f['ssp1']
    f.close()

    if (tname.lower().find('wheel')>-1):# | (tname.lower().find('circular')>-1):
        pos = np.array(Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['pos'])[0]

        ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
        f = np.load(ff, allow_pickle = True) 
        pcorr1 = f['pcorr1']
        pcorr2 = f['pcorr2']
        f.close()        
        pcorrmed = np.median(pcorr1)
        pcorr = pcorr1
        if np.median(pcorr2)> pcorrmed:
            pcorrmed = np.median(pcorr2)
            pcorr = pcorr1            

    else:
        xx0 = np.array(Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['x_pos'])[0]
        yy0 = np.array(Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['y_pos'])[0]
        pos = np.concatenate((xx0[:,np.newaxis], yy0[:, np.newaxis]),1)
    pos_all[sess_name0] = pos.copy()



num_neurons = len(pcorr)
numfigs = 4
numw = 5
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 300)
plt.viridis()
nw = 0
sig1 = 1
numbins1 = 30

torsort = np.flip(np.argsort(pcorr))



for nn, n in enumerate(torsort):
    nnn = nn%numh
    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
        nw += 1
    for sess_it, sess in enumerate(sess_names):
        sspk1 = spk_all[sess].copy()
        pos = pos_all[sess].copy()
        num_neurons = len(sspk1[0,:])
        cc = coords_aligned[sess].copy()%(2*np.pi)
        times = np.round(np.linspace(0, len(pos)-1, len(sspk1))).astype(int)
        pos = pos[times]

        ax = plt.subplot(gs2[nnn,sess_it]) 
        if sess_it == 0:
            mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(pos[:,0], pos[:,1],
                                          sspk1[:,n], statistic='mean', 
                                         bins=numbins1, range=None, expand_binnumbers=True)


            nans = np.isnan(mtot_tmp)
            mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
            nans = np.isnan(mtot_tmp)
            mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
            mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
            minval = 0 
            maxval = np.percentile(mtot_tmp.flatten(), 97.5)
#                mtot_tmp[nans] = -np.inf
            ax.imshow(mtot_tmp, origin = 'lower', vmin = minval, vmax = maxval)
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            binsx = np.linspace(0,np.max(pos)+1e-3, int(np.max(pos)/10))
            spk = binned_statistic(pos, sspk1[:,n], bins = binsx)[0]
            lencorr = 500
            spk[np.isnan(spk)] = 0
            acorrs = np.zeros(lencorr)

            spktemp = np.concatenate((spk, np.zeros(lencorr)))
            lenspk = len(spk)
            for t1 in range(lencorr):
                acorrs[t1] = np.sum(np.dot(spk,np.roll(spktemp, t1)[:lenspk]))
            acorrs /= acorrs[0]

            ax.bar(np.arange(lencorr), acorrs, width = 1, color = 'k')
            ax.set_xlim([0,lencorr])
            ax.set_ylim([0, 0.1])
            ax.set_xticks([0, 250, lencorr])
            ax.set_yticks([0.0, 0.05, 0.1])
            ax.set_yticklabels('')
            ax.set_xticklabels('')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.set_aspect(1/ax.get_data_ratio())

        ax = plt.subplot(gs2[nnn,sess_it+2]) 
        mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(cc[:,0], cc[:,1],
                                              sspk1[:,n], statistic='mean', 
                                             bins=numbins1, range=None, expand_binnumbers=True)
        nans = np.isnan(mtot_tmp)
        mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
        mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,0), numbins1+1, sig1, bClose = True) 
        minval = 0 
        maxval = np.percentile(mtot_tmp.flatten(), 97.5)
#            mtot_tmp[nans] = -np.inf
        ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], 
                  vmin = minval, vmax = maxval)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect(1/ax.get_data_ratio())
        r_box = transforms.Affine2D().skew_deg(15,15)
        for x in ax.images + ax.lines + ax.collections:
            trans = x.get_transform()
            x.set_transform(r_box+trans) 
            if isinstance(x, PathCollection):
                transoff = x.get_offset_transform()
                x._transOffset = r_box+transoff     
        ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
        ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
        ax.set_aspect('equal', 'box') 
        ax.axis('off')   
    df = pd.concat(data, ignore_index=True, axis=1)            
    df.columns = data_names
    df.to_excel('Source_data/Obenhaus_tuning/n'+ str(nn)+'.xlsx', sheet_name='neuron' + str(nn))  

plt.savefig('Figures/obenhaus_ratemaps_all.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/obenhaus_ratemaps_all.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)
plt.show()


In [ ]:
from matplotlib.collections import PathCollection
import cv2 as cv
numbins1 = 30
sig1 = 2
roll = 3
plt.viridis()
for icount, ii in enumerate([2]):
    msesscurr = metasessions[ii] 
    sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
    pos_all = {}
    spk_all = {}
    f = np.load('Results/' + msesscurr + '_coords_aligned.npz' , allow_pickle = True) 
    coords_aligned = f['coords_aligned'][()]
    f.close()
    


    for sess_name0 in sess_names[:]:
        tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]        
        ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
        f = np.load(ff, allow_pickle = True) 
        spk_all[sess_name0] = f['ssp1']
        f.close()

        if (tname.lower().find('wheel')>-1):# | (tname.lower().find('circular')>-1):
            pos = np.array(Tracking_Linear[Tracking_Linear['session_name'] == sess_name0]['pos'])[0]
            
            ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
            f = np.load(ff, allow_pickle = True) 
            pcorr1 = f['pcorr1']
            pcorr2 = f['pcorr2']
            f.close()        
            pcorrmed = np.median(pcorr1)
            pcorr = pcorr1
            if np.median(pcorr2)> pcorrmed:
                pcorrmed = np.median(pcorr2)
                pcorr = pcorr1            

        else:
            xx0 = np.array(Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['x_pos'])[0]
            yy0 = np.array(Tracking_OpenField[Tracking_OpenField['session_name'] == sess_name0]['y_pos'])[0]
            pos = np.concatenate((xx0[:,np.newaxis], yy0[:, np.newaxis]),1)
        pos_all[sess_name0] = pos.copy()

            

    num_neurons = 5
    numfigs = 4
    numw = 1
    numh = int(np.ceil(num_neurons/numw))
    outer1 = gridspec.GridSpec(1, numw)
    fig = plt.figure(figsize=(np.ceil((numw*numfigs+numw-1)*1.05), np.ceil(numh*1.1)), dpi = 300)
    plt.viridis()
    nw = 0
    sig1 = 1
    numbins1 = 30
                        
    torsort = np.flip(np.argsort(pcorr))

  
    
    for nn, n in enumerate(torsort[86:91]):
        data = []            
        data_names = []
        nnn = nn%numh
        if nnn == 0:
            outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = .3)
            gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.2,wspace = .0)
            nw += 1
        for sess_it, sess in enumerate(sess_names):
            sspk1 = spk_all[sess].copy()
            pos = pos_all[sess].copy()
            num_neurons = len(sspk1[0,:])
            cc = coords_aligned[sess].copy()%(2*np.pi)
            times = np.round(np.linspace(0, len(pos)-1, len(sspk1))).astype(int)
            pos = pos[times]
            
            ax = plt.subplot(gs2[nnn,sess_it]) 
            if sess_it == 0:
                mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(pos[:,0], pos[:,1],
                                              sspk1[:,n], statistic='mean', 
                                             bins=numbins1, range=None, expand_binnumbers=True)


                nans = np.isnan(mtot_tmp)
                mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
                nans = np.isnan(mtot_tmp)
                mtot_tmp[nans] = np.mean(mtot_tmp[~nans])
                mtot_tmp = gaussian_filter(mtot_tmp,sigma = sig1)
                minval = 0 
                maxval = np.percentile(mtot_tmp.flatten(), 97.5)
#                mtot_tmp[nans] = -np.inf
                ax.imshow(mtot_tmp, origin = 'lower', vmin = minval, vmax = maxval)
                ax.set_xticks([])
                ax.set_yticks([])
            else:
                binsx = np.linspace(0,np.max(pos)+1e-3, int(np.max(pos)/10))
                spk = binned_statistic(pos, sspk1[:,n], bins = binsx)[0]
                lencorr = 500
                spk[np.isnan(spk)] = 0
                acorrs = np.zeros(lencorr)

                spktemp = np.concatenate((spk, np.zeros(lencorr)))
                lenspk = len(spk)
                for t1 in range(lencorr):
                    acorrs[t1] = np.sum(np.dot(spk,np.roll(spktemp, t1)[:lenspk]))
                acorrs /= acorrs[0]

                ax.bar(np.arange(lencorr), acorrs, width = 1, color = 'k')
                ax.set_xlim([0,lencorr])
                ax.set_ylim([0, 0.1])
                ax.set_xticks([0, 250, lencorr])
                ax.set_yticks([0.0, 0.05, 0.1])
                ax.set_yticklabels('')
                ax.set_xticklabels('')
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                ax.set_aspect(1/ax.get_data_ratio())
            data.append(pd.Series(mtot_tmp.flatten()))
            data_names.extend(['spatial_tuning' + str(sess_it)])

            ax = plt.subplot(gs2[nnn,sess_it+2]) 
            mtot_tmp, x_edge, y_edge,c2 = binned_statistic_2d(cc[:,0], cc[:,1],
                                                  sspk1[:,n], statistic='mean', 
                                                 bins=numbins1, range=None, expand_binnumbers=True)
            nans = np.isnan(mtot_tmp)
            mtot_tmp[np.isnan(mtot_tmp)] = np.mean(mtot_tmp[~np.isnan(mtot_tmp)])
            mtot_tmp = smooth_tuning_map(np.rot90(mtot_tmp,0), numbins1+1, sig1, bClose = True) 
            minval = 0 
            maxval = np.percentile(mtot_tmp.flatten(), 97.5)
#            mtot_tmp[nans] = -np.inf
            ax.imshow(mtot_tmp, origin = 'lower', extent = [0,2*np.pi,0, 2*np.pi], 
                      vmin = minval, vmax = maxval)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_aspect(1/ax.get_data_ratio())
            r_box = transforms.Affine2D().skew_deg(15,15)
            for x in ax.images + ax.lines + ax.collections:
                trans = x.get_transform()
                x.set_transform(r_box+trans) 
                if isinstance(x, PathCollection):
                    transoff = x.get_offset_transform()
                    x._transOffset = r_box+transoff     
            ax.set_xlim(0, 2*np.pi + 3*np.pi/5)
            ax.set_ylim(0, 2*np.pi + 3*np.pi/5)
            ax.set_aspect('equal', 'box') 
            ax.axis('off')   
            
            
            data.append(pd.Series(mtot_tmp.flatten()))
            data_names.extend(['toroidal_tuning' + str(sess_it)])            
        df = pd.concat(data, ignore_index=True, axis=1)            
        df.columns = data_names
        df.to_excel('Source_data/Fig2e_obenhaus_tuning'+ str(nn)+'.xlsx', sheet_name='obenhaus_tuning')  
        
    plt.savefig('Figures/obenhaus_ratemaps.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
    plt.savefig('Figures/obenhaus_ratemaps.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)
    plt.show()


## Videos

In [ ]:
msesscurr = metasessions[-1] 
sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
pos_all = {}
spk_all = {}

sess_name0 = sess_names[0]
tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]        
ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
f = np.load(ff, allow_pickle = True) 
spk_all[sess_name0] = f['ssp1']
xx0 = f['xx0']
yy0 = f['yy0']
coords_ds_consistent = f['coords_ds']
movetimes0 = f['movetimes0']
indstemp = f['indstemp']
f.close()

sspk2 = spk0_all[sess_name0][:,mod_ind1s]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)
sspk2 = gaussian_filter1d(sspk2,sigma = sig, axis = 0)
sspk2 = np.sqrt(sspk2)                                


In [ ]:
starttime = 1000
num_frames = 1000
coordsnew_mod11,coordsbox1 = get_coords_distribution1(sspk2[movetimes0,:], 
                                                      sspk2, 1-coords_ds_consistent,
                                                      indstemp, 
                                                      starttime = starttime, num_frames = num_frames)
                                       

In [ ]:
######### Video without gain #########
times0 = np.round(np.linspace(0, len(xx0)-1, len(sspk2))).astype(int)

headpos1 = preprocessing.minmax_scale(np.concatenate((xx0[times0][:, np.newaxis],
                                                      yy0[times0][:, np.newaxis]),1))
coordsbox11 = (coordsbox1)*2*np.pi/3+2*np.pi/3

ttmp = 500

tt1 = tt0[times0].copy()
tt1[:starttime] = tt1[starttime]
plt.viridis()

fig = plt.figure(figsize = (8,5), dpi = 160)#,constrained_layout=True)
gs = fig.add_gridspec(1, 2, height_ratios = [1,],width_ratios = [1,1])

ax = fig.add_subplot(gs[0, 0])
img = ax.scatter([], [], zorder = -1, s = 20, c = 'k', alpha = 0.7)
img1 = ax.scatter([], [], c = 'r', zorder = -1, s = 50)
minh, maxh = 0, 1
ax.axis('off')

ax.set_xlim([minh-0.01, maxh+0.01])
ax.set_ylim([minh-0.01, maxh+0.01])
c1 = 0.2
c11 = [c1,c1,c1]
a, b,c,d = [0,0], [0,1], [1,0], [1,1] 
ax.plot([a[0], b[0]],[a[1], b[1]], c = c11, ls = '-')
ax.plot([a[0], c[0]],[a[1], c[1]], c = c11, ls = '-')
ax.plot([b[0], d[0]],[b[1], d[1]], c = c11, ls = '-')
ax.plot([c[0], d[0]],[c[1], d[1]], c = c11, ls = '-')
ax.set_aspect(1/ax.get_data_ratio())



ax2 = fig.add_subplot(gs[:, 1])
ax2.set_xlim( np.sin(np.pi/12)*2*np.pi, np.cos(np.pi/12)*2*np.pi)
ax2.set_ylim( np.sin(np.pi/12)*2*np.pi, np.cos(np.pi/12)*2*np.pi)
ax2.set_aspect('equal','box')
ax2.axis('off')
cc_comb = np.concatenate((coordsnew_mod11[0,:,:],coordsnew_mod11[0,:,:],coordsnew_mod11[0,:,:]),1)
cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)

r_box = transforms.Affine2D().skew_deg(15,15)


img2 = ax2.imshow(cc_comb, origin = 'lower',
                  extent = [0,2*np.pi, 0, 2*np.pi], vmin = 0, vmax = 10)
img3 = ax2.scatter([],[], c = [[0.7,0.7,0.7]], s = 5, alpha = 0.6, transform=r_box + ax2.transData)
img4 = ax2.scatter([],[], c = [[0.9, 0.2,0.2]], s = 15, alpha = 1, transform=r_box + ax2.transData)

ax2.plot([2/3*np.pi,2/3*np.pi],[2/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')
ax2.plot([2/3*np.pi, 4/3*np.pi],[2/3*np.pi,2/3*np.pi], c = [0.5,0.5,0.5], ls = '--')

ax2.plot([4/3*np.pi,4/3*np.pi],[2/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')
ax2.plot([2/3*np.pi, 4/3*np.pi],[4/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')


im = [img, img1, img2,img3, img4, ]
def init():
    im[0].set_offsets(np.array([[],[]]).T)
    im[1].set_offsets([[]])  
    
    im[2].set_data([[]])
    im[3].set_offsets(np.array([[],[]]).T)
    im[4].set_offsets([[]])  
    
    return im

for x in (ax2.images + ax2.lines + ax2.collections + ax2.texts):
    trans = x.get_transform()
    x.set_transform(r_box+trans) 
    if isinstance(x, PathCollection):
        transoff = x.get_offset_transform()
        x._transOffset = r_box+transoff     

fps = 25
height, width = 1600, 1600
ax2.set_title('Torus')
ax.set_title('Open Field')

def animate(i):
    i+= starttime  
    im[0].set_offsets(headpos1[starttime:i+1,:])
    im[1].set_offsets(headpos1[i:i+1,:])

    cc_comb = np.concatenate((coordsnew_mod11[i-starttime,:,:],
                              coordsnew_mod11[i-starttime,:,:],
                              coordsnew_mod11[i-starttime,:,:]),1)
    cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)

    im[2].set_data(cc_comb)    
    im[3].set_offsets(coordsbox11[starttime:i+1,:])    
    im[4].set_offsets(coordsbox11[i:i+1,:])    
    
    return im
anim = animation.FuncAnimation(fig, animate,# init_func=init,
                               frames=1000, interval=30, blit=True)

In [ ]:
anim.save('Results/Obenhaus_OF.mp4')

In [ ]:
msesscurr = metasessions[-1] 
sess_names = list(Session[Session['metasession_name']==msesscurr]['session_name'])
pos_all = {}
spk_all = {}

sess_name0 = sess_names[1]
tname = list(Session[Session['session_name']==sess_name0]['timeseries_name'])[0]        
ff = glob.glob('Results/' + '*_' + sess_name0 + '.npz')[0]
f = np.load(ff, allow_pickle = True) 
spk_all[sess_name0] = f['ssp1']
xx0 = f['xx0']
yy0 = f['yy0']
coords_ds_consistent = f['coords_ds']
movetimes0 = f['movetimes0']
indstemp = f['indstemp']
f.close()

sspk2 = spk0_all[sess_name0][:,mod_ind1s]
sspk2 = preprocessing.minmax_scale(sspk2, axis = 0)
sspk2 = gaussian_filter1d(sspk2,sigma = sig, axis = 0)
sspk2 = np.sqrt(sspk2)                                


In [ ]:
starttime = 1000
num_frames = 1000
coordsnew_mod11,coordsbox1 = get_coords_distribution1(sspk2[movetimes0,:], 
                                                      sspk2, 1-coords_ds_consistent,
                                                      indstemp, 
                                                      starttime = starttime, num_frames = num_frames)
                                       

In [ ]:
######### Video without gain #########
headpos1 = preprocessing.minmax_scale(xx0[times0])
coordsbox11 = (coordsbox1)*2*np.pi/3+2*np.pi/3
ys = [0, 560]

ttmp = 500

tt1 = tt0[times0].copy()
tt1[:starttime] = tt1[starttime]
plt.viridis()

fig = plt.figure(figsize = (8,5), dpi = 160)#,constrained_layout=True)
gs = fig.add_gridspec(2, 2, height_ratios = [3,1],width_ratios = [1,1.5])

ax = fig.add_subplot(gs[0, 0])
img, = ax.plot(tt1[:ttmp],headpos1[:ttmp], c = 'k')
minh, maxh = 0, 1
ax.set_ylim(minh, maxh)
ax.set_xlim( tt1[0], tt1[ttmp])
plt.gca().axes.spines['top'].set_visible(False)
plt.gca().axes.spines['right'].set_visible(False)


ax2 = fig.add_subplot(gs[:, 1])
ax2.set_xlim( np.sin(np.pi/12)*2*np.pi, np.cos(np.pi/12)*2*np.pi)
ax2.set_ylim( np.sin(np.pi/12)*2*np.pi, np.cos(np.pi/12)*2*np.pi)
ax2.set_aspect('equal','box')
ax2.axis('off')
cc_comb = np.concatenate((coordsnew_mod11[0,:,:],coordsnew_mod11[0,:,:],coordsnew_mod11[0,:,:]),1)
cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)

r_box = transforms.Affine2D().skew_deg(15,15)


img2 = ax2.imshow(cc_comb, origin = 'lower',
                  extent = [0,2*np.pi, 0, 2*np.pi], vmin = 0, vmax = 10)
img3 = ax2.scatter([],[], c = [[0.7,0.7,0.7]], s = 5, alpha = 0.6, transform=r_box + ax2.transData)
img4 = ax2.scatter([],[], c = [[0.9, 0.2,0.2]], s = 15, alpha = 1, transform=r_box + ax2.transData)

ax2.plot([2/3*np.pi,2/3*np.pi],[2/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')
ax2.plot([2/3*np.pi, 4/3*np.pi],[2/3*np.pi,2/3*np.pi], c = [0.5,0.5,0.5], ls = '--')

ax2.plot([4/3*np.pi,4/3*np.pi],[2/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')
ax2.plot([2/3*np.pi, 4/3*np.pi],[4/3*np.pi,4/3*np.pi], c = [0.5,0.5,0.5], ls = '--')


im = [img,  img2,img3, img4, ]
def init():
    im[0].set_data([],[])
    
    im[1].set_data([[]])
    im[2].set_offsets(np.array([[],[]]).T)
    im[3].set_offsets([[]])  
    
    return im

for x in (ax2.images + ax2.lines + ax2.collections + ax2.texts):
    trans = x.get_transform()
    x.set_transform(r_box+trans) 
    if isinstance(x, PathCollection):
        transoff = x.get_offset_transform()
        x._transOffset = r_box+transoff     

fps = 25
height, width = 1600, 1600
ax2.set_title('Torus')
ax.text(tt1[starttime]-ttmp*0.03, maxh/2, 'Wheel')

def animate(i):
    i+= starttime    
    if i-starttime < ttmp:
        im[0].set_data(tt1[starttime:i+1], headpos1[starttime:i+1])
        im[0].axes.axis([tt1[starttime], tt1[starttime+ttmp], minh, maxh])
        xs = [tt1[starttime], tt1[starttime+ttmp]]
    else:    
        im[0].set_data(tt1[i-ttmp:i+1], headpos1[i-ttmp:i+1])
        im[0].axes.axis([tt1[i-ttmp], tt1[i+1], minh, maxh])
        xs = [tt1[i-ttmp], tt1[i+1]]            
    ax.set_xticks(xs)
    ax.set_xticklabels([str(int(xi)) + 's' for xi in xs])
    ax.set_yticks([minh,maxh])
    ax.set_yticklabels([str(yi) + 'cm' for yi in ys])        
    cc_comb = np.concatenate((coordsnew_mod11[i-starttime,:,:],
                              coordsnew_mod11[i-starttime,:,:],
                              coordsnew_mod11[i-starttime,:,:]),1)
    cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)

    im[1].set_data(cc_comb)    
    im[2].set_offsets(coordsbox11[starttime:i+1,:])    
    im[3].set_offsets(coordsbox11[i:i+1,:])    
    
    return im
anim = animation.FuncAnimation(fig, animate,# init_func=init,
                               frames=1000, interval=30, blit=True)
#anim = animation.FuncAnimation(fig, animate, init_func=init,
#                               frames=10, interval=50, blit=True)


In [ ]:
anim.save('Results/Obenhaus_wheel.mp4')

In [ ]:
for diff in [120,]:
    ks = np.array([[0,0], [1,0], [0,1], [1,1], [-1,0], [0,-1], [-1,-1], [1,-1], [-1,1]])
    ij = starttime+diff    
    cctrial_temp =coordsbox1[ij-100:ij,:]*2*np.pi
    cctrial = np.zeros_like(cctrial_temp)
    cctrial[0,:] = cctrial_temp[0,:].copy()            
    k1, k2 = 0, 0
    for cn  in range(len(cctrial_temp)-1):
        ctmp1 = cctrial_temp[cn+1]
        c_temp = [ctmp1 + (k1*2*np.pi, k2*2*np.pi), 
                  ctmp1 + ((k1+1)*2*np.pi, k2*2*np.pi), 
                  ctmp1 + (k1*2*np.pi, (k2+1)*2*np.pi), 
                  ctmp1 + ((k1+1)*2*np.pi, (k2+1)*2*np.pi), 
                  ctmp1 + ((k1-1)*2*np.pi, k2*2*np.pi), 
                  ctmp1 + (k1*2*np.pi, (k2-1)*2*np.pi), 
                  ctmp1 + ((k1-1)*2*np.pi, (k2-1)*2*np.pi), 
                  ctmp1 + ((k1+1)*2*np.pi, (k2-1)*2*np.pi), 
                  ctmp1 + ((k1-1)*2*np.pi, (k2+1)*2*np.pi), 
                 ]  
        cmin = np.argmin(np.sum(np.square(c_temp-cctrial[cn]),1))
        cctrial[cn+1,:] = c_temp[cmin]
        k1 += ks[cmin][0]
        k2 += ks[cmin][1]          







    #num_rhomb = int(np.max(cctrial)/(2*np.pi))+1
    num_rhomb = 3
    fig = plt.figure(figsize = (20,10))    

    plt.axis('off')        
    ax = fig.add_subplot(111)          

    cc_comb = np.concatenate((coordsnew_mod11[ij-starttime,:,:],coordsnew_mod11[ij-starttime,:,:],coordsnew_mod11[ij-starttime,:,:]),1)
    cc_comb = np.concatenate((cc_comb,cc_comb,cc_comb),0)
    ax.imshow(cc_comb, origin = 'lower',
                      extent = [0,2*np.pi*num_rhomb, 0, 2*np.pi*num_rhomb], vmin = 0, vmax = 20, zorder = -4)

    ax.scatter(cctrial[:,0], cctrial[:,1], 
               s = 60, alpha = 1, c = preprocessing.minmax_scale(range(len(cctrial[:,0]))), 
               vmin = -0.4,vmax = 1.6, cmap = 'hot', zorder = -1, transform=r_box + ax.transData)
    ax.scatter(cctrial[-1,0], cctrial[-1,1], 
               s = 150, alpha = 1, c = 'k' ,
               vmin = 0, vmax = 1, marker = 'X', zorder = -1, transform=r_box + ax.transData)

    for i in range(num_rhomb):
        for j in range(num_rhomb):
            ax.plot([2*np.pi*i,2*np.pi*(i+1)],[2*np.pi*j,2*np.pi*j], c = [0.6,0.6,0.6], ls = '--', lw = 1.5, zorder = -2)
            ax.plot([2*np.pi*i,2*np.pi*i],[2*np.pi*j,2*np.pi*(j+1)], c = [0.6,0.6,0.6], ls = '--', lw = 1.5, zorder = -2)
    i += 1
    for j in range(i):
        ax.plot([2*np.pi*i,2*np.pi*i],[2*np.pi*j,2*np.pi*(j+1)], c = [0.6,0.6,0.6], ls = '--', lw = 1.5, zorder = -2)
    j += 1
    for i in range(j):
        ax.plot([2*np.pi*i,2*np.pi*(i+1)],[2*np.pi*j,2*np.pi*j], c = [0.6,0.6,0.6], ls = '--', lw = 1.5, zorder = -2)


    ax.set_aspect(1/ax.get_data_ratio())
    r_box = transforms.Affine2D().skew_deg(15,15)
    for x in ax.images + ax.lines + ax.collections:
        trans = x.get_transform()
        x.set_transform(r_box+trans) 
        if isinstance(x, PathCollection):
            transoff = x.get_offset_transform()
            x._transOffset = r_box+transoff     
    ax.set_xlim(-np.pi, (num_rhomb)*2*np.pi + 8*3*np.pi/5)
    ax.set_ylim(-np.pi, (num_rhomb)*2*np.pi + 8*3*np.pi/5)
    ax.set_aspect('equal', 'box') 
    ax.axis('off')
    fig.tight_layout(h_pad = 0, w_pad = 0)
                     
                     
    data = []
    data_names = []
    data.append(pd.Series(cc_comb.flatten()))
    data_names.extend(['tuning_map',])
    data.append(pd.Series(cctrial[:,0]))
    data_names.extend(['path_x'])
    data.append(pd.Series(cctrial[:,1]))
    data_names.extend(['path_y'])
    df = pd.concat(data, ignore_index=True, axis=1)            
    df.columns = data_names
    df.to_excel("Source_data/Fig1d_torus.xlsx", sheet_name='torus_traj')  

    plt.savefig('Figures/torus_traj.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
    plt.savefig('Figures/torus_traj.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)


In [ ]:
headpos1 = preprocessing.minmax_scale(xx0[times0])
coordsbox11 = (coordsbox1)*2*np.pi/3+2*np.pi/3
ys = [0, 560]

ttmp = 500

tt1 = tt0[times0].copy()
tt1[:starttime] = tt1[starttime]

fig = plt.figure(figsize = (8,5), dpi = 120)
ax = fig.add_subplot(121)
#img = ax.scatter([], [], zorder = -1)
#img1 = ax.scatter([], [], c = 'r', zorder = -1)
img, = ax.plot(tt1[:100],headpos1[:100], c = 'k')
minh, maxh = 0, 1
img.set_data(tt1[ij-100:ij+1], headpos1[ij-100:ij+1])
ax.scatter(tt1[ij-100:ij+1], headpos1[ij-100:ij+1], cmap = 'hot', 
           c  = preprocessing.minmax_scale(range(101)), alpha = 1, s = 50, vmin = -0.4,vmax = 1.6)
ax.scatter(tt1[ij], headpos1[ij], c = 'k', marker = 'X', alpha = 1, s = 100)

img.axes.axis([tt1[ij-100-2], tt1[ij+3], minh, maxh])
ax.axis('off') 
ax.set_aspect(1/ax.get_data_ratio())

data = []
data_names = []
data.append(pd.Series(tt1[ij-100:ij+1]))
data_names.extend(['time',])
data.append(pd.Series(headpos1[ij-100:ij+1]))
data_names.extend(['wheel',])
df = pd.concat(data, ignore_index=True, axis=1)            
df.columns = data_names
df.to_excel("Source_data/Fig1d_wheel.xlsx", sheet_name='wheel_traj')  

plt.savefig('Figures/wheel_traj.png', transparent = True, bbox_inches='tight', pad_inches=0.2)
plt.savefig('Figures/wheel_traj.pdf', transparent = True, bbox_inches='tight', pad_inches=0.2)
